In [61]:
import cv2
import numpy as np
from math import *

undistorted_image = cv2.imread("000001.jpg")
undistorted_image = cv2.resize(undistorted_image, (undistorted_image.shape[1], undistorted_image.shape[0]), interpolation=cv2.INTER_CUBIC)
undistorted_image.shape

(900, 1600, 3)

In [62]:
def PointMap(xx, yy, r):
    ''' 
    将一个鱼眼图像上的点用等距模型映射到单位球面
    input:
        x,y: 输入参数，畸变图像上的点的坐标
        theta_max 鱼眼镜头视场角
        r  制作的鱼眼图像半径
    output:
        new_x, new_y : 输出参数，单位球面上点的坐标
    '''

    l = np.sqrt(np.power(xx, 2) + np.power(yy, 2))    # 鱼眼图上某点距中心距离
    theta_max = pi / 2   # 鱼眼镜头半场视角

    alpha = np.ones_like(xx) * (pi/2)
    alpha[xx != 0] = np.arctan2(yy[xx != 0], xx[xx != 0]) 
    
    f = r / theta_max   # 用等距投影的方式计算焦距f
    theta = l / f       # 鱼眼图映射到单位圆（theta = r/f）
    d = f * np.tan(theta)  # 单位圆映射到原图

    tx = d * np.cos(alpha)
    ty = d * np.sin(alpha)

    # 防止图片出现撕裂
    tx[xx > 0] = np.abs(tx[xx > 0])
    tx[xx < 0] = - np.abs(tx[xx < 0])
    tx[xx == 0] = 0
    ty[yy > 0] = np.abs(ty[yy > 0])
    ty[yy < 0] = - np.abs(ty[yy < 0])
    ty[yy == 0] = 0


    return tx, ty

In [63]:

def RectifyMap(r):
    '''
    生成从原图像到鱼眼图像的坐标的映射矩阵mapx mapy  
    input:
	    r: 圆半径，鱼眼图像半径
    output:
        mapy, mapy: 映射关系
    '''
    width = 2 * r   # 映射图像的宽度
    height = 2 * r
    center_x = width / 2
    center_y = height / 2

    x = np.linspace(0, width-1, num=width, dtype=int)
    y = np.linspace(0, height-1, num=height, dtype=int)
    centx = np.ones_like(x, dtype=int) * center_x
    centy = np.ones_like(y, dtype=int) * center_y
    x = x - centx
    y = centy - y
    xx, yy = np.meshgrid(x, y)

    inv_mask = np.sqrt(np.power(xx, 2) + np.power(yy, 2)) > r   # 鱼眼成像范围随便映射为原图像尺寸外即可
    mapx, mapy = PointMap(xx, yy, r)
    mapx[inv_mask] = -1e5
    mapy[inv_mask] = -1e5
    
    return mapx, mapy

mapx, mapy = RectifyMap(100)

In [64]:
def UndisImage(undistorted_image, mapx, mapy):
    '''
    矫正图像
    '''
    assert(mapx.shape == mapy.shape)
    height, width = undistorted_image.shape[0:2]
    channel = undistorted_image.shape[-1]
    cx = width / 2
    cy = height / 2

    x = (mapx + np.ones_like(mapx) * cx)
    y = (np.ones_like(mapy) * cy - mapy)
    print(x)
    
    distort_image = cv2.remap(undistorted_image, x.astype(np.float32), y.astype(np.float32), cv2.INTER_LINEAR)

    return distort_image

distort_image = UndisImage(undistorted_image, mapx, mapy)

cv2.imwrite('fisheye.jpg', distort_image)

[[-99200. -99200. -99200. ... -99200. -99200. -99200.]
 [-99200. -99200. -99200. ... -99200. -99200. -99200.]
 [-99200. -99200. -99200. ... -99200. -99200. -99200.]
 ...
 [-99200. -99200. -99200. ... -99200. -99200. -99200.]
 [-99200. -99200. -99200. ... -99200. -99200. -99200.]
 [-99200. -99200. -99200. ... -99200. -99200. -99200.]]


True